## Week4-Assignment
뉴스 기사를 분류해보기!
word2vec, doc2vec 등 아무거나 사용해도 되고,
분류 자체도 코사인유사도, PCA, random forest, 두개로 나뉘니까 logistic regression 등 편하신 코드를 사용하시면 될 것 같습니다!

```
하수의 코드니까
코드는 자유롭게 수정해도 좋습니다!
```

In [ ]:
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

# 기본 글꼴 변경
import matplotlib as mpl
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumBarunGothic')

!pip install soynlp
!pip install konlpy
!pip install wordcloud
!pip install soynlp



     |████████████████████████████████| 419kB 16.2MB/s 
     |████████████████████████████████| 19.4MB 7.1MB/s 
     |████████████████████████████████| 92kB 12.5MB/s 
     |████████████████████████████████| 460kB 51.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


### A. 한글 기사

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import sklearn

In [ ]:
def get_breaking_link():
    url = "https://search.naver.com/search.naver?"

    url_list = []
    #크롤링할 페이지수 
    for i in range(0,30):
      params = {
          # 탭
          "where": 'news',

          # 검색어
          "query": '속보',

          # 페이지
          "start": i,
      }

      response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)
      soup = BeautifulSoup(response.text, "html.parser")


      for i in soup.select('div.info_group > a'):
          if re.search('news.naver.com', i['href']):
              url_list.append(i['href'])

    return url_list

def get_link(query):
    url = "https://search.naver.com/search.naver?"

    url_list = []
    #크롤링할 페이지수 
    for i in range(0,1):
      params = {
          # 탭
          "where": 'news',

          # 검색어
          "query": query,

          # 페이지
          "start": i,
      }

      response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, params=params)
      soup = BeautifulSoup(response.text, "html.parser")


      for i in soup.select('div.info_group > a'):
          if re.search('news.naver.com', i['href']):
              url_list.append(i['href'])

    return url_list


def get_article(url_list):

    title_list = []
    content_list = []
    for url in url_list:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        try:
          document_content = BeautifulSoup(response.content, "html.parser")
        except Exception as e:
          print(f"crawling failed for url:{url}, Error: {e}")
          continue  
          
        # 기사 제목 가져옴
        tag_headline = document_content.find_all('h3', {'id': 'articleTitle'}, {'class': 'tts_head'})
        text_headline = ''  # 뉴스 기사 제목 초기화
        text_sentence = ''  # 뉴스 기사 본문 초기화
        try:
          # 기사  제목 예시 : [<h3 id="articleTitle">]제목내용</h3>]
          # 필요 없는 내용 날리고 깔끔하게 처리해주기
          text_headline = text_headline + str(tag_headline[0].find_all(text=True))
          if not text_headline:  # 공백일 경우 기사 제외 처리
            continue
          # 기사 본문 가져옴
          tag_content = document_content.find_all('div', {'id': 'articleBodyContents'})
          text_sentence = text_sentence + str(tag_content[0].find_all(text=True))
          if not text_sentence:  # 공백일 경우 기사 제외 처리
            continue
          title_list.append(text_headline)
          content_list.append(text_sentence)
        except Exception as e: 
          print('fail')
    return title_list, content_list

In [ ]:
#검색어를 입력해주세요
query1= input("검색어(한글)를 입력해주세요: ")
news_link1 = get_link(query1)
print(news_link1)
print()
news_title1, news_content1 = get_article(news_link1)
#제목이랑 내용이랑 같은 숫자인지 확인
len(news_title1) , len(news_content1)

검색어(한글)를 입력해주세요: 테슬라
['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004608883', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0012285250', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=028&aid=0002537730', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=215&aid=0000946769', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=003&aid=0010413089', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=009&aid=0004769957', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=092&aid=0002217298', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=009&aid=0004769785', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=028&aid=0002537798']



(9, 9)

In [ ]:
#속보를 가져옵니다
news_link2 = get_breaking_link()
print(news_link2)
print()
news_title2, news_content2 = get_article(news_link2)
#제목이랑 내용이랑 같은 숫자인지 확인
len(news_title2) , len(news_content2)

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=015&aid=0004520192', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=028&aid=0002537906', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010415142', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=011&aid=0003888841', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=082&aid=0001078184', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=009&aid=0004770089', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=005&aid=0001424531', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003604397', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005249239', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=629&aid=0000074210', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=015&aid=0004520192', 'https://news.naver.

(300, 300)

In [ ]:
#두개의 내용을 섞어줍니다

segment = []
news_title = news_title1 + news_title2
news_content = news_content1 + news_content2
for i in range(len(news_title1)):
  segment.append(query1)
for i in range(len(news_title2)):  
  segment.append('속보')

In [ ]:
def title_preprocessing(text):
    val_split = text.split(sep='flash_removeCallback() {}\n\n')
    val_split
    text = text.replace('무단 전재 재배포 금지','')
    text = text.replace("\\\\n", "")
    text = text.replace("\\n", "")
    text = text.replace("\n", "")
    text = text.replace("\\", "")
    text = text.replace("\\t", "")
    text = text.replace("NaN", "")
    text = text.replace("  ", " ")
    text = text.replace("\t", "")

    # 한글, english 빼고 전부 날리기
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z|0-9|.|]', ' ', text)
    text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》{}_▲▶兆]', '', text)
    return text

def content_preprocessing(text):
    val_split = text.split(sep='flash_removeCallback()')
    text = val_split[1]
    text = text.replace('무단 전재 재배포 금지','')
    text = text.replace("\\\\n", "")
    text = text.replace("\\n", "")
    text = text.replace("\n", "")
    text = text.replace("\\tt", "")
    text = text.replace("\tt", "")
    text = text.replace("\t", "")
    text = text.replace("\\", "")
    text = text.replace("\\t", "")
    text = text.replace("NaN", "")
    text = text.replace("  ", " ")
    text = text.replace("\t", "")

    # 한글, english 빼고 전부 날리기
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z|0-9|.|]', ' ', text)
    text = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》{}_▲▶兆]', '', text)
    return text

news_list_1 = []
news_list_2 = []

for i in news_title:
    news_list_1.append(title_preprocessing(i))

for i in news_content:    
    news_list_2.append(content_preprocessing(i))

['  머스크  테슬라  미 최대 기업될 수 있어   ',
 '   비트코인으로 테슬라 차 사면 세금폭탄 가능성   ',
 '  비트코인으로 테슬라 사면 소득세   ',
 '  테슬라  제동  전통완성차  가속 ...전기차 판도 바뀌나  ',
 '  올해만  130조원 거래  통 커진 서학개미  테슬라  1위  ',
 '  미중 고래싸움에 등터진 서학개미  테슬라 어쩌나   ',
 '  현대차 vs 테슬라 빠른 전기차 충전 인프라 대결  ',
 '   테슬라 게 섰거라  춘추전국시대 전기차 시장  ',
 '  테슬라  차량 영상 수집한다는데 국내 보안시설은 무방비  ',
 '   속보  농심 창업주 신춘호 회장 별세 향년 92세  ',
 '  코로나 신규확진자 505명 36일 만에 500명대로  ',
 '   속보 신규 확진 505명  36일만에 500명대 국내발생 490명  ',
 '   속보 농심 창업주 신춘호 회장 별세   향년 92세  ',
 '   속보  한강서 고양 발달장애인 추정 시신 발견 산책 중 실종  ',
 '   속보  코로나19 어제 505명 신규확진 36일 만에 다시 500명대로  ',
 '   속보   신라면 신화  쓴 농심 창업주 신춘호 회장 별세  ',
 '   속보   라면왕  농심 창업주 신춘호 회장 별세  ',
 '   속보 신규 확진 505명  36일만에 500명대 지역발생 확진 490명  ',
 '   속보  코로나 신규확진 505명 36일 만에 500명 넘어  ',
 '   속보  농심 창업주 신춘호 회장 별세 향년 92세  ',
 '  코로나 신규확진자 505명 36일 만에 500명대로  ',
 '   속보 신규 확진 505명  36일만에 500명대 국내발생 490명  ',
 '   속보 농심 창업주 신춘호 회장 별세   향년 92세  ',
 '   속보  한강서 고양 발달장애인 추정 시신 발견 산책 중 실종  ',
 '   속보  코로나19 어제 505명 신규확진 36일 만에 다시 500명대로  ',
 '   속보  

In [ ]:
df_merge = pd.DataFrame(list(zip(segment,news_list_1, news_list_2)),
              columns=['segment','title','content'])
#df_merge=sklearn.utils.shuffle(df_merge)
df_merge = df_merge.reset_index()
df_merge['index'] = df_merge['index'].astype(str)
df_merge.head()

,index,segment,title,content
0,285,속보,속보 비수도권 34.6 26일 오후 6시까지 전국 381명 확진,서울 107 강원 30 부산 29 충북 16 대구 13 경...
1,53,속보,속보 리병철 미사일 발사는 자위권 미 대통령 발언 도발,좌측은 리병철 당중앙위원회 비서 우측은 조 바이든 미국 대통령...
2,180,속보,속보 북한 어제 신형전술유도탄 시험발사 김정은 불참,25일 오후 서울 수서역 대합실에서 시민들이 북한 탄도미사일 추...
3,206,속보,속보 한강서 고양 발달장애인 추정 시신 발견,지난달 28일 경기 고양시 평화누리길 행주산성둘레길에서 실종된 ...
4,177,속보,속보 홍남기 투기근절대책 내주초 발표 도심 후보지도 공개,사진 뉴스1 홍남기 부총리 겸 기획재정부 장관은 26일 ...


In [ ]:
df_merge.tail()

,index,segment,title,content
304,293,속보,속보 바이든 긴장 고조 선택하면 대응 외교도 준비,첫 공식 기자회견서 미사일 발사 질문에 답변 조 바이든...
305,82,속보,속보 북한 리병철 미사일 발사는 자위권 미국 대통령 발언은 도발,북한이 미사일 시험발사는 주권국가의 자위권에 해당한다며 조 바...
306,232,속보,속보 홍남기 투기근절대책 내주초 발표 도심 후보지도 공개,사진 뉴스1 홍남기 부총리 겸 기획재정부 장관은 26일 ...
307,203,속보,속보 코로나 신규 확진자 505명... 36일만에 500명대,26일 오후 서울 영등포구 여의도 윤중로에서 시민들이 산책을 즐...
308,12,속보,속보 농심 창업주 신춘호 회장 별세 향년 92세,신춘호 회장. 사진제공 농심 서울경제 라면왕...


In [ ]:
#word2vec 후 백터의 평균값 구하기 혹은 dec2vec 임베딩 이후 아래 코드를 참고하셔서, 시각화하는 것이 과제입니다! 
#시각화 하실 때 검색어의 segment와 속보의 segment를 구분하면 좋을 것 같아요!
#아래 코드는 준교님이 진행하신 실습 https://colab.research.google.com/drive/1SNZM9IaUO7QXMsNKbFCH0V12jxAvTNL1#scrollTo=Mn4Y6ueQPa8y 에서 가져왔어요
#이외의 부분은 참고하시면 될 것 같아요
#유의미하게 학습하려면 기사 20000개는 필요할 것 같은데 너무 오래결러서 

# >>>>> word2vec 임베딩 해서 시각화만 하는걸로 하고, 위에 언급한 부분은 하고 싶은 것만 하시면 될 것 같아요.

In [ ]:
from sklearn.manifold import TSNE      # t-Stochastic Neighbor Embedding
from sklearn.decomposition import PCA  # Principal Component Analysis 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import pandas as pd 
from gensim.models import KeyedVectors 

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처 
mpl.rcParams['axes.unicode_minus'] = False 
plt.rc('font', family='D2Coding') 

def show_tsne(): 
  tsne = TSNE(n_components=2) 
  X = tsne.fit_transform(X_show) 
  df = pd.DataFrame(X, index=vocab_show, columns=['x', 'y']) 
  fig = plt.figure() 
  fig.set_size_inches(30, 20) 
  ax = fig.add_subplot(1, 1, 1) 
  ax.scatter(df['x'], df['y']) 
  for word, pos in df.iterrows(): 
    ax.annotate(word, pos, fontsize=10) 
  plt.xlabel("t-SNE 특성 0") 
  plt.ylabel("t-SNE 특성 1") 
  plt.show() 

def show_pca(): # PCA 모델을 생성합니다 
  pca = PCA(n_components=2) 
  pca.fit(X_show) # 처음 두 개의 주성분으로 숫자 데이터를 변환합니다 
  x_pca = pca.transform(X_show) 
  plt.figure(figsize=(30, 20)) 
  plt.xlim(x_pca[:, 0].min(), x_pca[:, 0].max()) 
  plt.ylim(x_pca[:, 1].min(), x_pca[:, 1].max()) 
  for i in range(len(X_show)): 
    plt.text(x_pca[i, 0], x_pca[i, 1], str(vocab_show[i]), fontdict={'weight': 'bold', 'size': 9}) 
  plt.xlabel("첫 번째 주성분") 
  plt.ylabel("두 번째 주성분") 
  plt.show() 

#model_name = '/content/vectors.txt' 
#model = KeyedVectors.load_word2vec_format(model_name) 
vocab = list(model.wv.vocab) 
X = model[vocab] 
# n 개의 단어에 대해서만 시각화 
n = 800 
X_show = X[:n,:] 
vocab_show = vocab[:n] 
show_tsne() 
show_pca()